In [8]:
# ================================================================================
# STEP 1: Setup and Imports
# ================================================================================
# Import all required libraries and set up the environment

# Core CrewAI imports
from crewai import Agent, Crew, Task, Process, LLM
from crewai.tools import BaseTool
from crewai_tools import SerperDevTool

# Pydantic for structured outputs
from pydantic import BaseModel, Field
from typing import List, Type, Optional

# Environment and utilities
import os
import json
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables from .env file
load_dotenv()

# Verify API keys are set
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️ Warning: OPENAI_API_KEY not found in environment")
if not os.getenv('SERPER_API_KEY'):
    print("⚠️ Warning: SERPER_API_KEY not found in environment")

print("✅ Libraries imported successfully!")

# Initialize the LLM
llm = LLM(model="gpt-4")
print("✅ LLM initialized (GPT-4)")

✅ Libraries imported successfully!
✅ LLM initialized (GPT-4)


In [ ]:

# --- Structured Outputs with Pydantic models ---

class LearningMaterial(BaseModel):
    title: str
    url: Optional[str] = None
    type: str  # 'video', 'article', 'exercise'
    description: Optional[str] = None

class LearningMaterialsOutput(BaseModel):
    materials: List[LearningMaterial] = Field(description="A list of curated learning materials")

class QuizQuestion(BaseModel):
    question: str
    choices: List[str]
    answer: str

class QuizOutput(BaseModel):
    questions: List[QuizQuestion]

class ProjectIdea(BaseModel):
    title: str
    description: str
    expertise_level: str  # 'beginner', 'intermediate', 'advanced'

class ProjectSuggestionsOutput(BaseModel):
    projects: List[ProjectIdea]

# --- Custom Tool: ProjectSuggestionTool ---

class ProjectSuggestionInput(BaseModel):
    expertise_level: str
    topics: List[str]

class ProjectSuggestionTool(BaseTool):
    name: str = "Project Suggestion Tool"
    description: str = "Suggests practical project ideas based on expertise level and topics of interest."
    args_schema: Type[BaseModel] = ProjectSuggestionInput

    def _run(self, expertise_level: str, topics: List[str]) -> str:
        # Here you can integrate with a smart backend or just return a mock suggestion
        joined_topics = ', '.join(topics)
        if expertise_level.lower() == 'beginner':
            desc = f"A simple project for learning: Create a flashcard app to review {joined_topics}."
        elif expertise_level.lower() == 'intermediate':
            desc = f"An intermediate project: Build a blog engine that lets users post and quiz each other about {joined_topics}."
        else:
            desc = f"An advanced project: Implement a full-stack personalized learning platform specializing in {joined_topics} topics."
        idea = ProjectIdea(title=f"{expertise_level.title()} {joined_topics} Project", description=desc, expertise_level=expertise_level)
        return ProjectSuggestionsOutput(projects=[idea]).json()

# --- User Input Setup ---

USER_TOPICS = ["Python programming"]
USER_EXPERTISE = "intermediate"

# --- Agents ---

learning_material_agent = Agent(
    llm=LLM(model="gpt-4"),
    role="Learning Material Agent",
    backstory="Expert in curating engaging learning materials across subject domains using web search tools and best resources.",
    goal="Curate personalized learning materials (videos, articles, exercises) for users based on their topics of interest. Respond with a structured Pydantic output.",
    verbose=True,
    tools=[SerperDevTool()],
    output_schema=LearningMaterialsOutput,
)

quiz_creator_agent = Agent(
    llm=LLM(model="gpt-4"),
    role="Quiz Creator Agent",
    backstory="A creative quiz-maker who helps reinforce user learning by generating tailored questions.",
    goal="Generate appropriate quiz questions (multiple choice) to test understanding of learning materials. Structure responses as Pydantic models.",
    verbose=True,
    output_schema=QuizOutput
)

project_idea_agent = Agent(
    llm=LLM(model="gpt-4"),
    role="Project Idea Agent",
    backstory="Inventive assistant who suggests practical project ideas to cement learning and build real-world skills.",
    goal="Propose project ideas based on the user's expertise level and topics. Response is structured as a ProjectSuggestionsOutput Pydantic model.",
    verbose=True,
    tools=[ProjectSuggestionTool()],
    output_schema=ProjectSuggestionsOutput
)

# --- Tasks ---

learning_material_task = Task(
    description=(
        f"Curate a list of videos, articles, and exercises for the user on these topics: {', '.join(USER_TOPICS)}. "
        "Use web search tools to find a balance of engaging and authoritative resources. The output should be a list of LearningMaterial objects in a Pydantic LearningMaterialsOutput."
    ),
    agent=learning_material_agent,
    expected_output="A structured list of learning materials (videos, articles, exercises).",
    output_schema=LearningMaterialsOutput
)

quiz_task = Task(
    description=(
        "Given the curated learning materials, generate 5-7 quiz questions (multiple choice) to assess the user's understanding of the above topics. "
        "Return as a list of QuizQuestion objects using the QuizOutput schema."
    ),
    agent=quiz_creator_agent,
    expected_output="A structured quiz with questions, choices, and answers.",
    output_schema=QuizOutput
)

project_task = Task(
    description=(
        f"Suggest practical project ideas for a user whose expertise level is '{USER_EXPERTISE}' and whose topics of interest are {', '.join(USER_TOPICS)}. "
        "Project suggestions should help the user apply their knowledge. Use the Project Suggestion Tool. Respond with a ProjectSuggestionsOutput."
    ),
    agent=project_idea_agent,
    expected_output="A structured list of suggested projects.",
    output_schema=ProjectSuggestionsOutput
)

# --- Compose Crew and Run Sequentially ---

crew = Crew(
    agents=[learning_material_agent, quiz_creator_agent, project_idea_agent],
    tasks=[learning_material_task, quiz_task, project_task],
    process=Process.sequential,
    verbose=True
)

results = crew.kickoff()


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4e608288-cff3-4914-a9a3-fcb83bb3445b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Learning Material Agent                                                                                 │
│                                                                                                                 │
│  Task: Curate a list of videos, articles, and exercises for the user on these topics: Python programming. Use   │
│  web search tools to find a balance of engaging and authoritative resources. The output should be a list of     │
│  LearningMaterial objects in a Pydantic LearningMaterialsOutput.                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Learning Material Agent                                                                                 │
│                                                                                                                 │
│  Thought: I need to find a variety of resources for learning Python programming. I will start by searching for  │
│  video tutorials on the internet.                                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6e66be52-b438-45a4-bb1d-9d5fded60808                                                                     │
│  Agent: Project Idea Agent                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow code                                   │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file               │
│  • Run: crewai traces enable                                                 │
│                                                                              │
╰─────────────────────────

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 4e608288-cff3-4914-a9a3-fcb83bb3445b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Here are some intermediate Python programming project suggestions based on your interests:       │
│                                                                                                                 │
│  1. Project Title: Intermediate Python Programming Project                                                      │
│     Description: Build a blog engine that lets users post and quiz each other about Python programming. This    │
│  project will help boost your understanding of Python and its real-world applications. It's a practical and     │
│  effective way to get hands-on experience and deepen your programming skills.                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [21]:
1

1